In [3]:
import numpy as np

In [49]:
class MMboard:
    '''An instance of a MasterMind board.
    
    The object of the game is to guess the hidden code of length L, 
    where each code element is any one of C characters. 
    
    In response to a guess of the code, if the code is incorrect,
    a response (b, w) is provided; b represents the number of 
    characters where the correct character is provided in the 
    correct position, whereas w represents the number of characters
    that are in the code but are in the wrong position (i.e. the
    number of characters that are actually in the code but are not
    in the position guessed). 
    
    The game continues until the correct code is guessed, or until
    a predefined number of allowable tries have been exhausted. '''
    
    def __init__ (self, codelength=4, numcolors=6, maxtries=10, suppress_output=False):
        self._L = codelength
        self._C = numcolors
        self._N_iters = maxtries
        self._nooutput = suppress_output
        
        self._code = np.zeros(self._L)  # contains the code
        
        self.n_guessed = 0  # number of guesses tried
        self.gameover = False
    
    def _codeOK(self, cc):
        '''Helper function to check that inputs for code are in
        proper form. '''
        
        try:
            init = np.array([int(item) for item in cc])
        except:
            raise ValueError('Code not in the form of a list/array of length ' + str(self._L))

        if len(init) != self._L:
            raise ValueError('Code not of length ' + str(self._L))

        if not all(isinstance(item, int) for item in init):
            raise ValueError('Each character must be an integer, between 0 and ' + str(self._C - 1))

        if not((init >= 0).all() and (init <= self._C - 1).all()):
            raise ValueError('Each integer must be between 0 and ' + str(self._C - 1)) 
        
        return True
        
    def set_code(self, custom=None, showcode=True):
        '''Initalize/reset the code.
        
        Can set the code manually (provide a list or array of length L)
        or set one at random (default).
        
        Example: myboard.set_code([0, 1, 2, 3])
        
        showcode: whether or not the code being set is displayed
        suppress_output: do not print messages
        '''
        
        if (custom is not None) and self._codeOK(custom):  
            self._code = np.array([int(item) for item in custom])
        else:
            self._code = np.random.randint(0, self._C, size=self._L)  # each element is [0, C)
        
        if showcode and (not self._nooutput):
            print "Code successfully initialized to ", self._code
        elif not self._nooutput:
            print "Code successfully initialized. Good luck."
            
        self.n_guessed = 0  # reset guess counter
        self.gameover = False
        
    def _check_guess(self, guess):
        '''Process a guess.
        
        Take a guess in the form of a list of integers, and returns
        (number of characters in the correct position,
         number of characters in the wrong position but elsewhere in code)
        
        Example: myboard.guess_code([0, 1, 2, 3])
        Returns: (1, 1)  # code is [2 1 4 4] 
        
        Note: None is returned if an error is encountered.'''
        
        try:
            assert self._codeOK(guess)
            self.n_guessed += 1
            if not self._nooutput:
                print "guess #" + str(self.n_guessed) + " of " + str(self._N_iters) + ": you guessed ", guess     
        except:
            if not self._nooutput:
                print "  please enter a list of %i integers from 0 to %i. try again" % (self._L, self._C)
            return None
        
        # counters
        corpos = 0  
        wrongpos = 0  
        # track non-counted characters
        code_left = []
        guess_left = []
       
        # check for correct digit in correct place  
        for i, digit in enumerate(self._code):            
            if digit==guess[i]:
                corpos += 1  
            else:
                code_left.append(digit)
                guess_left.append(guess[i])
            
        assert len(code_left) == len(guess_left)

        if len(code_left)>0:
            # check for correct digit in wrong place
            for digit in code_left:
                if digit in guess_left:
                    wrongpos += 1
                    guess_left.remove(digit) # removes only one occurrence
        
        if not self._nooutput:                
            print "you have %i right item(s) in the right place, and %i right item(s) but in the wrong place" % (corpos, wrongpos)         
        
        return corpos, wrongpos

    def guess_code(self, guess):
        '''Entry method to process guess, with a check to see if number of guesses has
        been exceeded, or code guessed correctly.
        
        If game is over, self.gameover is set to True.
        
        Returns: a tuple - (number of characters in the correct position,
                            number of characters in the wrong position but elsewhere in code); 
                  None if there was an error in the guess; or
                  -1 if game is over'''
        
        if not self.gameover:
            try:
                b, w = self._check_guess(guess)
                assert (b + w) <= self._L
                
                if b == self._L:
                    if not self._nooutput:
                        print "You win!"
                    self.gameover = True
                elif self.n_guessed == self._N_iters:
                    if not self._nooutput:
                        print "Game over. The correct code was", self._code  
                    self.gameover = True
                    
                return (b, w)
            except:
                pass  # error in guess
                return None
        
        return -1

In [73]:
a = MMboard()
# a = MMboard(suppress_output=True)

In [74]:
a.set_code([2, 1, 4, 4])

Code successfully initialized to  [2 1 4 4]


In [75]:
a.guess_code([0, 1, 2, 4])

guess #1 of 10: you guessed  [0, 1, 2, 4]
you have 2 right item(s) in the right place, and 1 right item(s) but in the wrong place


(2, 1)

In [76]:
a.guess_code([0, 3, 2, 4])

guess #2 of 10: you guessed  [0, 3, 2, 4]
you have 1 right item(s) in the right place, and 1 right item(s) but in the wrong place


(1, 1)

In [77]:
a.guess_code([0, 3, 2, 9])

  please enter a list of 4 integers from 0 to 6. try again


In [78]:
a.guess_code([1, 1, 1, 1])

guess #3 of 10: you guessed  [1, 1, 1, 1]
you have 1 right item(s) in the right place, and 0 right item(s) but in the wrong place


(1, 0)

In [79]:
a.guess_code([4, 4, 4, 4])

guess #4 of 10: you guessed  [4, 4, 4, 4]
you have 2 right item(s) in the right place, and 0 right item(s) but in the wrong place


(2, 0)

In [80]:
a.guess_code([1, 2, 4, 4])

guess #5 of 10: you guessed  [1, 2, 4, 4]
you have 2 right item(s) in the right place, and 2 right item(s) but in the wrong place


(2, 2)

In [81]:
a.guess_code([2, 1, 4, 4])

guess #6 of 10: you guessed  [2, 1, 4, 4]
you have 4 right item(s) in the right place, and 0 right item(s) but in the wrong place
You win!


(4, 0)

In [82]:
a.guess_code([1, 1, 4, 4]) # game already over

-1

In [83]:
a.set_code([1, 1, 3, 3])

Code successfully initialized to  [1 1 3 3]


In [84]:
a.guess_code([0, 0, 0, 0])

guess #1 of 10: you guessed  [0, 0, 0, 0]
you have 0 right item(s) in the right place, and 0 right item(s) but in the wrong place


(0, 0)

In [85]:
a.guess_code([0, 0, 0, 0])
a.guess_code([0, 0, 0, 0])
a.guess_code([0, 0, 0, 0])
a.guess_code([0, 0, 0, 0])
a.guess_code([0, 0, 0, 0])
a.guess_code([0, 0, 0, 0])
a.guess_code([0, 0, 0, 0])
a.guess_code([0, 0, 0, 0])
a.guess_code([0, 0, 0, 0])

guess #2 of 10: you guessed  [0, 0, 0, 0]
you have 0 right item(s) in the right place, and 0 right item(s) but in the wrong place
guess #3 of 10: you guessed  [0, 0, 0, 0]
you have 0 right item(s) in the right place, and 0 right item(s) but in the wrong place
guess #4 of 10: you guessed  [0, 0, 0, 0]
you have 0 right item(s) in the right place, and 0 right item(s) but in the wrong place
guess #5 of 10: you guessed  [0, 0, 0, 0]
you have 0 right item(s) in the right place, and 0 right item(s) but in the wrong place
guess #6 of 10: you guessed  [0, 0, 0, 0]
you have 0 right item(s) in the right place, and 0 right item(s) but in the wrong place
guess #7 of 10: you guessed  [0, 0, 0, 0]
you have 0 right item(s) in the right place, and 0 right item(s) but in the wrong place
guess #8 of 10: you guessed  [0, 0, 0, 0]
you have 0 right item(s) in the right place, and 0 right item(s) but in the wrong place
guess #9 of 10: you guessed  [0, 0, 0, 0]
you have 0 right item(s) in the right place, and

(0, 0)

In [244]:
b = MMboard()
b.set_code(showcode=False)

Code successfully initialized. Good luck.


In [245]:
b.guess_code([0, 0, 0, 0])

guess #1 of 10: you guessed  [0, 0, 0, 0]
you have 0 right item(s) in the right place, and 0 right item(s) but in the wrong place


In [246]:
b.guess_code([1, 1, 1, 1])

guess #2 of 10: you guessed  [1, 1, 1, 1]
you have 1 right item(s) in the right place, and 0 right item(s) but in the wrong place


In [247]:
b.guess_code([2, 2, 2, 2])

guess #3 of 10: you guessed  [2, 2, 2, 2]
you have 2 right item(s) in the right place, and 0 right item(s) but in the wrong place


In [248]:
b.guess_code([3, 3, 3, 3])

guess #4 of 10: you guessed  [3, 3, 3, 3]
you have 0 right item(s) in the right place, and 0 right item(s) but in the wrong place


In [249]:
b.guess_code([4, 4, 4, 4])

guess #5 of 10: you guessed  [4, 4, 4, 4]
you have 0 right item(s) in the right place, and 0 right item(s) but in the wrong place


In [250]:
b.guess_code([1, 2, 2, 5])

guess #6 of 10: you guessed  [1, 2, 2, 5]
you have 1 right item(s) in the right place, and 3 right item(s) but in the wrong place


In [251]:
b.guess_code([5, 1, 2, 2])

guess #7 of 10: you guessed  [5, 1, 2, 2]
you have 2 right item(s) in the right place, and 2 right item(s) but in the wrong place


In [252]:
b.guess_code([5, 2, 1, 2])

guess #8 of 10: you guessed  [5, 2, 1, 2]
you have 4 right item(s) in the right place, and 0 right item(s) but in the wrong place
You win!
